# Using tensorflow to create `Convolutional Neural Network`(`CNN`) with `fashion_mnist` example

**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=tensorflow&ea=CNN_fashion_mnist_without_keras&dt=CNN_fashion_mnist_without_keras)

Using `CNN` will make it easier to process image classification. By using `tensorflow` without `keras` version to implement a very simple, `CNN` will help readers to understand `tensorflow`.


## Import Modules

In [1]:
# !pip install tensorflow numpy

In [2]:
import tensorflow as tf
import numpy as np
from  PIL import Image

## Load Datasets
use `tf.keras.datasets` to complete the data import (Use keras may have faster network speed). 

In [3]:
(images_train, labels_train), (images_test, labels_test) = tf.keras.datasets.fashion_mnist.load_data()

In [4]:
Image.fromarray(images_test[1]).save("test.png")

In [5]:
images_train = images_train / 255.0
images_test = images_test / 255.0
# For an ordinary picture, there are usually 3 dimensions, of which the last dimension is 3, 
# by useing the grayscale image as an example, so only one dimension is expanded, and the last dimension is 1.
images_train = np.reshape(images_train, (60000, 28, 28, 1))
images_test = np.reshape(images_test, (10000, 28, 28, 1))
# print("images train shape:{}\nimages test shape:{}".format(images_train.shape, labels_train.shape))

## Define the layers 
Generally speaking, we use the following three operations:
- [`conv2d`](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) with `padding="SAME"` and performing Leaky ReLU activation function
- [`maxpool`](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool2d) by `2 x 2`
- [`dense`](https://www.tensorflow.org/api_docs/python/tf/nn/leaky_relu) perform Leaky ReLU activation function

In [6]:
leaky_relu_alpha = 0.2
dropout_rate = 0.5 

# conv2d
def conv2d(inputs, filters, stride_size):
    out = tf.nn.conv2d(inputs, filters, strides=[1, stride_size, stride_size, 1], padding="SAME") 
    return tf.nn.leaky_relu(out, alpha=leaky_relu_alpha) 
# max pool
def maxpool(inputs, pool_size, stride_size):
    return tf.nn.max_pool2d(inputs, ksize=[1, pool_size, pool_size, 1], padding='VALID', strides=[1, stride_size, stride_size, 1])

# Leaky ReLU
def dense(inputs, weights):
    x = tf.nn.leaky_relu(tf.matmul(inputs, weights), alpha=leaky_relu_alpha)
    return tf.nn.dropout(x, rate=dropout_rate)

In [7]:
# targets
output_classes = 10
# Glorot uniform initializer to initialize weithts
initializer = tf.initializers.glorot_uniform()
def get_weight(shape, name):
    return tf.Variable(initializer(shape), name=name, trainable=True, dtype=tf.float32 )

# shapes of each layer operation
shapes = [
    [3, 3, 1, 4], # conv2d 28 x 28  x 4 
    [3, 3, 4, 4], # conv2d 28 x 28  x 4
    # maxpool  14 x 14 x 4 
    [784, 128], # dense 784 to 128
    [128, output_classes], # 128 to 10
]

## Model Configuration
In this case, the operations includes   
`convolution layer 1(60000x28x28x4)` $\rightarrow$ `convolution layer 2(60000x28x28x4)`   
$\rightarrow$ `max pooling(60000x14x14x4)` $\rightarrow$ `fully connected layer(60000x784)`   
$\rightarrow$ `fully connected layer(60000x128)` $\rightarrow$ `fully connected layer(60000x10)`  
Finally get the results.

In [8]:
class Model(tf.Module):
    # def __init__(self, in_features, out_features, name=None):
    def __init__(self):
        # # initialize the weights
        self.weights=[]
        for i in range(len(shapes)):
            self.weights.append(get_weight(shapes[i], "weight{}".format(i)))
    
    @tf.function(input_signature=[tf.TensorSpec(shape=(None, 28, 28, 1), dtype=tf.uint8, name='x')])
    def __call__(self, x):
        x = tf.cast(x, dtype=tf.float32) 
        # convolution layer 1
        c1 = conv2d(x,self.weights[0], stride_size=1) 
        # convolution layer 2
        c2 = conv2d(c1, self.weights[1], stride_size=1)
        # maxpool
        p2 = maxpool(c2, pool_size=2, stride_size=2)
        # flatten the output into a vector
        flatten = tf.reshape(p2, shape=(tf.shape(p2)[0], -1))
        # fully connected layer
        d3 = dense(flatten, self.weights[2])
        logits = tf.matmul(d3, self.weights[3])
        # return the last layer
        return tf.nn.softmax(logits)
model = Model()

In [9]:
# loss function
def loss(pred, target):
    return tf.losses.categorical_crossentropy(target, pred)

# accuracydefining model
def accur(pred, target):
    pred_true = tf.argmax(pred, axis=1)
    y_true = tf.argmax(target, axis=1)
    y_eq = tf.cast(tf.equal(pred_true, y_true), tf.float16)
    return tf.reduce_mean(y_eq)

learning_rate = 0.001
optimizer = tf.optimizers.Adam(learning_rate)

#training operartions
def train_step(model, inputs, outputs):
    with tf.GradientTape() as tape:
        pred = model(inputs)
        current_loss = loss(pred, outputs)
    grads = tape.gradient(current_loss, model.weights)
    optimizer.apply_gradients(zip(grads, model.weights))
    #print(tf.reduce_mean(current_loss).numpy(), accur(pred, outputs))
    return accur(pred, outputs).numpy()

## Train the model

In [10]:
num_epochs = 10
batch_size = 100


y = tf.one_hot(labels_train, depth=10)
X = images_train
for e in range(num_epochs):
    accurs = []
    for b in range(0, len(X), batch_size): 
        acc = train_step(model, X[b: b + batch_size], y[b: b + batch_size])
        accurs.append(acc)
    print(np.mean(accurs))

0.3984
0.438
0.4478
0.4563
0.4568
0.4614
0.4636
0.4663
0.4683
0.4697


# Create BentoService class

## install BentoML

In [11]:
# !pip install bentoml

In [12]:
%%writefile CNN_fashion_mnist_without_keras.py

import numpy as np
import tensorflow as tf
import bentoml

from bentoml.artifact import TensorflowSavedModelArtifact
from bentoml.adapters import ImageInput

FASHION_MNIST_CLASSES = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


@bentoml.env(pip_dependencies=['tensorflow', 'numpy', 'pillow'])
@bentoml.artifacts([TensorflowSavedModelArtifact('model')])
class FashionMnistTensorflow(bentoml.BentoService):
    @bentoml.api(input=ImageInput(pilmode="L"), batch=True)
    def predict(self, inputs):
        X = tf.constant(inputs, dtype=tf.uint8)
        X = tf.expand_dims(X, -1)
        outputs = self.artifacts.model(X)
        output_classes = tf.math.argmax(outputs, axis=1)
        return [FASHION_MNIST_CLASSES[c] for c in output_classes]

Overwriting CNN_fashion_mnist_without_keras.py


In [13]:
import CNN_fashion_mnist_without_keras
import bentoml
import importlib


importlib.reload(CNN_fashion_mnist_without_keras)
importlib.reload(bentoml)

service = CNN_fashion_mnist_without_keras.FashionMnistTensorflow()
service.pack("model", model)
service.save()

[2020-11-06 17:50:10,242] WARNING - Importing from "bentoml.artifact.*" has been deprecated. Instead, use`bentoml.frameworks.*` and `bentoml.service.*`. e.g.:, `from bentoml.frameworks.sklearn import SklearnModelArtifact`, `from bentoml.service.artifacts import BentoServiceArtifact`, `from bentoml.service.artifacts.common import PickleArtifact`
[2020-11-06 17:50:10,294] WARNING - pip package requirement tensorflow already exist
INFO:tensorflow:Assets written to: /tmp/bentoml-temp-ljk0o6pa/FashionMnistTensorflow/artifacts/model_saved_model/assets
[2020-11-06 17:50:11,702] INFO - BentoService bundle 'FashionMnistTensorflow:20201106175010_C46220' saved to: /home/ruhan/bentoml/repository/FashionMnistTensorflow/20201106175010_C46220


'/home/ruhan/bentoml/repository/FashionMnistTensorflow/20201106175010_C46220'

In [14]:
# test localy
import imageio
img_array = imageio.imread('test.png')
inputs = [img_array]

service.predict(inputs)

['Trouser']

## Use BentoService with BentoML CLI

In [15]:
!bentoml get FashionMnistTensorflow

BENTO_SERVICE                                 AGE                           APIS                                  ARTIFACTS                            LABELS
FashionMnistTensorflow:20201106175010_C46220  0.84 seconds                  predict<ImageInput:DefaultOutput>     model<TensorflowSavedModelArtifact>
FashionMnistTensorflow:20201106173151_5F9336  18 minutes and 19.53 seconds  predict<ImageInput:DefaultOutput>     model<TensorflowSavedModelArtifact>
FashionMnistTensorflow:20201106172745_5D67B2  22 minutes and 25.88 seconds  predict<ImageInput:DefaultOutput>     model<TensorflowSavedModelArtifact>
FashionMnistTensorflow:20201106171319_1EB882  36 minutes and 52.19 seconds  predict<ImageInput:DefaultOutput>     model<TensorflowSavedModelArtifact>
FashionMnistTensorflow:20201106170537_541DC5  44 minutes and 33.47 seconds  predict<ImageInput:DefaultOutput>     model<TensorflowSavedModelArtifact>
FashionMnistTensorflow:20201106165520_2C2D52  54 minutes and 50.64 seconds  predict<ImageInp

**`bentoml get <BentoService name>:<bentoService version>` display detailed information of the specific BentoService version**

In [16]:
!bentoml get FashionMnistTensorflow:latest 

[2020-11-06 17:50:14,508] INFO - Getting latest version FashionMnistTensorflow:20201106175010_C46220
{
  "name": "FashionMnistTensorflow",
  "version": "20201106175010_C46220",
  "uri": {
    "type": "LOCAL",
    "uri": "/home/ruhan/bentoml/repository/FashionMnistTensorflow/20201106175010_C46220"
  },
  "bentoServiceMetadata": {
    "name": "FashionMnistTensorflow",
    "version": "20201106175010_C46220",
    "createdAt": "2020-11-06T09:50:11.676366Z",
    "env": {
      "condaEnv": "name: bentoml-default-conda-env\nchannels:\n- conda-forge\n- defaults\ndependencies:\n- pip\n",
      "pythonVersion": "3.7.9",
      "dockerBaseImage": "bentoml/model-server:0.9.2-py37",
      "pipPackages": [
        "bentoml==0.9.2",
        "tensorflow==2.3.1",
        "numpy==1.18.5",
        "pillow==8.0.0",
        "imageio==2.9.0"
      ]
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "TensorflowSavedModelArtifact"
      }
    ],
    "apis": [
      {
        "na

**Serve bentoml REST server locally**

In [17]:
# !bentoml serve FashionMnistTensorflow:latest

## Query REST API with python

In [18]:
import requests

with open("test.png", "rb") as f:
    img_bytes = f.read()


headers = {"Content-Type": "image/png"}

json_response = requests.post(f'http://127.0.0.1:5000/predict', data=img_bytes, headers=headers)
print(json_response)
print(repr(json_response.text))

<Response [200]>
'"Trouser"'


# reference
- [Image Classification From Scratch  With TensorFlow 2.0](https://colab.research.google.com/drive/15QZMSIrvE4MgVq2GnQ0XUsqQssY6sn7w#scrollTo=8T1Pf4mbuVRI&forceEdit=true&sandboxMode=true)*